# Tweem.io Similarity Comparison

### Imports

In [2]:
#  Necessary to allow assembly framework to run
import os
os.chdir('..')

In [72]:
import pandas as pd
import numpy as np
import inspect
import importlib

import assembly
from assembly import config as asmbl_config
from assembly import db as asmbl_db
from assembly import models as asmbl_models

from lib.tweemio import twm
from lib.tweemio import similarity
from lib.tweemio import twitter

In [73]:
importlib.reload(similarity)

<module 'lib.tweemio.similarity' from '/Users/liangjh/Workspace/tweemio-api/lib/tweemio/similarity.py'>

### Settings

In [4]:
os.environ['ASSEMBLY_ENV'] = 'Development'
app = assembly.Assembly.init(__name__, {'default': []})

### Calculate Similarity to Group

In [43]:
results = twm.calculate(app.config, screen_name='barackobama', group='trumpian') #, compare_to_screen_names=['realdonaldtrump'])

Evaluating similarity to realdonaldtrump with model: mdl_multinomialnbvect_v1
Evaluating similarity to kellyannepolls with model: mdl_multinomialnbvect_v1
Evaluating similarity to secpompeo with model: mdl_multinomialnbvect_v1
Evaluating similarity to mike_pence with model: mdl_multinomialnbvect_v1
Evaluating similarity to devinnunes with model: mdl_multinomialnbvect_v1
Evaluating similarity to lindseygrahamsc with model: mdl_multinomialnbvect_v1
Evaluating similarity to donaldjtrumpjr with model: mdl_multinomialnbvect_v1
Evaluating similarity to rudygiuliani with model: mdl_multinomialnbvect_v1
Evaluating similarity to govmikehuckabee with model: mdl_multinomialnbvect_v1
Evaluating similarity to alandersh with model: mdl_multinomialnbvect_v1
Evaluating similarity to repmattgaetz with model: mdl_multinomialnbvect_v1
Evaluating similarity to jim_jordan with model: mdl_multinomialnbvect_v1
Evaluating similarity to repmarkmeadows with model: mdl_multinomialnbvect_v1
Evaluating similarity 

In [44]:
{screen_name : np.mean(result.y_prob) for screen_name, result in results.items()}

{'realdonaldtrump': 0.0362871456636854,
 'kellyannepolls': 0.05737222991201928,
 'secpompeo': 0.11167948068663866,
 'mike_pence': 0.06720481314569705,
 'devinnunes': 1.9921557356344103e-05,
 'lindseygrahamsc': 0.007312821960141586,
 'donaldjtrumpjr': 0.0015974980293346611,
 'rudygiuliani': 0.0007456474545912521,
 'govmikehuckabee': 0.00879048543646449,
 'alandersh': 0.00044566331079554106,
 'repmattgaetz': 0.0010570736682972686,
 'jim_jordan': 0.0046734620896466015,
 'repmarkmeadows': 0.0002013814121971472,
 'gopleader': 0.008936633656353447,
 'gtconway3d': 0.00020415448715250007,
 'scaramucci': 0.01527626348948547}

In [83]:
res = results['mike_pence']

In [62]:
res = res[res.y_prob > 0.75].sort_values('y_prob', ascending=False)

In [63]:
res.y_prob

102    0.999631
97     0.998127
105    0.996245
18     0.990319
40     0.981850
59     0.795553
0      0.785098
Name: y_prob, dtype: float64

In [85]:
np.mean(res.y_prob)

0.06720481314569705

In [94]:
res.iloc[:3,:].to_dict('records')

[{'text': 'Americans across the country agree that we need bold action in the fight against climate change. #ActOnClimate  Show your support for bold action to fight climate change.  #ActOnClimate Adding clean energy jobs is another way the U.S. is continuing to #ActOnClimate.  President Obama has done more to improve energy efficiency than any previous U.S. leader.  #ActOnClimate',
  'y_prob': 0.7850975001360624,
  'y_sn': 'mike_pence'},
 {'text': "Editorial boards from across the country have called on Senate leaders to fill the Supreme Court vacancy.  Senate leaders are hamstringing our Supreme Court. This obstruction is unprecedented—and unacceptable:  #DoYourJob Judge Garland has now waited 146 days. That's shameful—and irresponsible. #DoYourJob  We've accomplished so much as a movement—and it's thanks to the power of organizing. ",
  'y_prob': 0.0001823461133651533,
  'y_sn': 'mike_pence'},
 {'text': "Don't let the cynics win. Keep pushing for change with @OFA.  Grab one of these

In [64]:
list(res.text)

["Michelle and I send our condolences to the people of New Zealand. We grieve with you and the Muslim community. All of us must stand against hatred in all its forms. In 2011, I visited the tiny town of Moneygall and got to walk around in the house where my great-great-great grandfather Falmouth Kearney lived his early life. I'll always be grateful for the warmth and generosity of the Irish people. Happy St. Patrick's Day! -Barack O’Bama  The floods in the Midwest and in southern Africa are heartbreaking. Suffering and loss on two sides of the world calls for solidarity and strength from all of us. Ask what you can do to help our fellow Americans and families in need. Here are some ideas:  And here are some ways to help cyclone victims in Mozambique, Malawi, and Zimbabwe: ",
 'As we celebrate Black History Month and Jackie Robinson’s 100th birthday, we celebrate the life of all whose courage opened the gates for everybody, and in the process, made America better.  We lost two great Ame

### Scratchpad

In [68]:
screen_name='barackobama'

In [66]:
tapi = twitter.TwitterApi(app.config['TWITTER_API_CREDS'])
tweets = tapi.timeline(screen_name, condense_factor=app.config['TWEET_CONDENSE_FACTOR'])

In [74]:
nlp_parser = similarity.get_nlp_readability()

nlpdocs = [nlp_parser(tweet) for tweet in tweets]

fkgl  = [nlpdoc._.flesch_kincaid_grade_level for nlpdoc in nlpdocs]
fkre  = [nlpdoc._.flesch_kincaid_reading_ease for nlpdoc in nlpdocs]
dc    = [nlpdoc._.dale_chall for nlpdoc in nlpdocs]
cl    = [nlpdoc._.coleman_liau_index for nlpdoc in nlpdocs]
autor = [nlpdoc._.automated_readability_index for nlpdoc in nlpdocs]

{
    'flesch_kincaid_grade_level': np.mean(fkgl),
    'flesch_kincaid_reading_ease': np.mean(fkre),
    'dale_chall': np.mean(dc),
    'coleman_liau_index': np.mean(cl),
    'automated_readability_index': np.mean(autor)
}